In [1]:
# =======================================================================================
# PROBLEM A5
#
# Build and train a neural network model using the Sunspots.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from kaggle.com/robervalt/sunspots
#
# Desired MAE < 0.15 on the normalized dataset.
# ========================================================================================

In [6]:
#Check for GPU support, tf and python ver
import tensorflow as tf

print(tf.__version__)
print(tf.test.is_built_with_gpu_support())
import sys
print(sys.version)

2.13.0
True
3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]


In [7]:
import csv
import numpy as np
import urllib

In [8]:
# Adjust performance and make series as tf.dataset
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [21]:
# Get dataset
data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
urllib.request.urlretrieve(data_url, 'sunspots.csv')

# Get data of sunspots and time_step from the dataset
time_step = []; sunspots = []
with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        sunspots.append(row[2])
        time_step.append(row[0])

In [24]:
series=np.array(sunspots, dtype=float)

In [26]:
# Normalization Function. DO NOT CHANGE THIS CODE
min=np.min(series)
max=np.max(series)
series -= min
series /= max
time=np.array(time_step)

In [32]:
# DO NOT CHANGE THIS CODE
split_time=3000

# splitting
time_train =  time[:split_time]
x_train =  series[:split_time]
time_valid =  time[split_time:]
x_valid = series[split_time:]

In [33]:
#Const
window_size=30
batch_size=32
shuffle_buffer_size=1000

# Get train_ds
train_set=windowed_dataset(x_train, window_size=window_size,
                            batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

2024-02-16 00:31:12.057055: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-16 00:31:12.259192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-16 00:31:12.259256: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-16 00:31:12.267529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-16 00:31:12.267627: I tensorflow/compile

In [57]:
class thecustomcallbacks(tf.keras.callbacks.Callback):
    def __init__(self):
        super(thecustomcallbacks, self).__init__()
    
    def on_epoch_end(self, epoch, logs=None):
        loggedtrain = logs["mae"]
        if loggedtrain < 0.15:
            self.model.stop_training = True

In [55]:
def get_model():
    model=tf.keras.Sequential([
        tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(None, 1)),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, "relu")        #Regression
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MSE, metrics = "mae")
    
    return model

In [56]:
model=get_model()
model.fit(train_set, epochs=10, callbacks=[thecustomcallbacks()])

Epoch 1/10
93/93 [==============================] - 5s 16ms/step - loss: 0.0108 - mae: 0.0744


# Answers!

In [61]:
# =======================================================================================
# PROBLEM A5
#
# Build and train a neural network model using the Sunspots.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from kaggle.com/robervalt/sunspots
#
# Desired MAE < 0.15 on the normalized dataset.
# ========================================================================================

import csv
import tensorflow as tf
import numpy as np
import urllib

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  series = tf.expand_dims(series, axis=-1)
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size + 1))
  ds = ds.shuffle(shuffle_buffer)
  ds = ds.map(lambda w: (w[:-1], w[1:]))
  return ds.batch(batch_size).prefetch(1)


def solution_A5():
  data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
  urllib.request.urlretrieve(data_url, 'sunspots.csv')

  time_step = []
  sunspots = []
  with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      sunspots.append(row[2])
      time_step.append(row[0])

  series = np.array(sunspots, dtype=float)

  # Normalization Function. DO NOT CHANGE THIS CODE
  min=np.min(series)
  max=np.max(series)
  series -= min
  series /= max
  time=np.array(time_step)

  # DO NOT CHANGE THIS CODE
  split_time=3000

  time_train =  time[:split_time]
  x_train =  series[:split_time]
  time_valid =  time[split_time:]
  x_valid = series[split_time:]

  # DO NOT CHANGE THIS CODE
  window_size=30
  batch_size=32
  shuffle_buffer_size=1000

  train_set=windowed_dataset(x_train, window_size=window_size,
                              batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

  class thecustomcallbacks(tf.keras.callbacks.Callback):
      def __init__(self):
          super(thecustomcallbacks, self).__init__()
      
      def on_epoch_end(self, epoch, logs=None):
          loggedtrain = logs["mae"]
          if loggedtrain < 0.15:
              self.model.stop_training = True
            
  def get_model():
      model=tf.keras.Sequential([
          tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(None, 1)),
          tf.keras.layers.LSTM(64),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(1, "relu")        #Regression
      ])
      model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MSE, metrics = "mae")
      
      return model
  
  model=get_model()
  model.fit(train_set, epochs=10, callbacks=[thecustomcallbacks()])
  
  return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_A5()
    model.save("model_A5.h5")


Epoch 1/10
93/93 [==============================] - 4s 16ms/step - loss: 0.0108 - mae: 0.0747


2024-02-16 00:50:48.640088: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 12407606590902358939
/root/miniconda3/envs/conda3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
